### Authenticate and get headers

In [ ]:
import requests
from _includes import config
from _includes import helper
import pandas as pd

config_dict = config.config_dict

api = helper.APIHelper(config_dict)
base_site_url = f'{api.server}/api/{api.api_version}/sites/{api.site_id}'
base_api_url = f'{api.server}/api/-'
headers = api.headers

### List all of metric definitions (with Helper Class)

In [ ]:
definition_dict = api.get_pulse_metric_definitions_dict()
definition_dict_lookup = {value: key for key, value in definition_dict.items()}

definition_dict_filter = {key: value for key, value in definition_dict.items() if 'DataDev' in key}
definition_df = pd.DataFrame(list(definition_dict_filter.items()), columns=['definition_name', 'definition_id'])
definition_df

### Loop Metrics for Each Metric Definition

In [ ]:
metric_list = []

for key, value in definition_dict_filter.items():
    print(f"Key: {key}, Value: {value}")
    definition_id = value
    endpoint = f'{base_api_url}/pulse/definitions/{definition_id}/metrics'
    api_response = requests.get(endpoint, headers=headers)
    api_response_json = api_response.json()
    metric_list.extend(api_response_json['metrics'])
    
    # Handle pagination
    while 'next_page_token' in api_response_json and api_response_json['next_page_token'] != '':
        next_page_token = api_response_json['next_page_token']
        api_response = requests.get(f'{endpoint}?pageToken={next_page_token}', headers=headers)
        api_response_json = api_response.json()
        metric_list.extend(api_response_json['metrics'])
        
print(metric_list)

### Transforming the JSON data into a structured format

In [ ]:
metric_data = []

for metric in metric_list:
    metric_data.append({
        'definition_id': metric['definition_id'],
        'definition_name': definition_dict_lookup.get(metric['definition_id']),
        'metric_id': metric['id'],
        'granularity': metric['specification']['measurement_period']['granularity'],
        'range': metric['specification']['measurement_period']['range'],
        'comparison': metric['specification']['comparison']['comparison']
    })

metric_df = pd.DataFrame(metric_data)
metric_df